In [1]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, root_mean_squared_error

import json

logical_cores = os.cpu_count()
print(f"Number of logical CPU cores: {logical_cores}")

num_workers = max(1, logical_cores // 2)
print(f"Number of workers set to: {num_workers}")

def is_gpu_available():
    try:
        return torch.cuda.is_available()
    except ImportError:
        return False

gpu_available = is_gpu_available()
print(f"GPU available: {gpu_available}")

torch.autograd.set_detect_anomaly(True)

Number of logical CPU cores: 16
Number of workers set to: 8
GPU available: True


In [2]:
os.makedirs('../models/hyperparameters-search-models/pytorch/conv1d-classification/', exist_ok=True)
os.makedirs('../models/hyperparameters-search-models/pytorch/conv1d-regression/', exist_ok=True)
os.makedirs('../models/hyperparameters-search-models/pytorch/lstm-classification/', exist_ok=True)
os.makedirs('../models/hyperparameters-search-models/pytorch/lstm-regression/', exist_ok=True)
os.makedirs('../models/hyperparameters-search-models/pytorch/transformer-classification/', exist_ok=True)
os.makedirs('../models/hyperparameters-search-models/pytorch/transformer-regression/', exist_ok=True)

os.makedirs('../models/best-hyperparameters/pytorch/conv1d-classification/', exist_ok=True)
os.makedirs('../models/best-hyperparameters/pytorch/conv1d-regression/', exist_ok=True)
os.makedirs('../models/best-hyperparameters/pytorch/lstm-classification/', exist_ok=True)
os.makedirs('../models/best-hyperparameters/pytorch/lstm-regression/', exist_ok=True)
os.makedirs('../models/best-hyperparameters/pytorch/transformer-classification/', exist_ok=True)
os.makedirs('../models/best-hyperparameters/pytorch/transformer-regression/', exist_ok=True)

os.makedirs('../models/trained-models/pytorch/conv1d-classification/', exist_ok=True)
os.makedirs('../models/trained-models/pytorch/conv1d-regression/', exist_ok=True)
os.makedirs('../models/trained-models/pytorch/lstm-classification/', exist_ok=True)
os.makedirs('../models/trained-models/pytorch/lstm-regression/', exist_ok=True)
os.makedirs('../models/trained-models/pytorch/transformer-classification/', exist_ok=True)
os.makedirs('../models/trained-models/pytorch/transformer-regression/', exist_ok=True)

path = '../data/train'
ticker_list = []

if os.path.exists(path):
    ticker_list = [os.path.splitext(f)[0] for f in os.listdir(path) if f.endswith('.csv')]


In [3]:
def load_or_create_ticker_df(csv_file_path):
    """
    Load the existing ticker DataFrame from a CSV file if it exists,
    otherwise create a new DataFrame with predefined column types.
    Ensure the DataFrame has the specified columns, add any missing columns,
    and rearrange the columns in alphabetical order, excluding 'Ticker_Symbol'.

    Args:
    csv_file_path (str): The path to the CSV file.

    Returns:
    pd.DataFrame: The loaded or newly created DataFrame.
    """
    # Define the column types
    column_types = {
        "Ticker_Symbol": str,
        "Best_Cov1D_Classification_Accuracy": float,
        "Best_Cov1D_Classification_Path": str,
        "Best_Cov1D_Regression_RMSE": float,
        "Best_Cov1D_Regression_Path": str,
        "Best_LSTM_Classification_Accuracy": float,
        "Best_LSTM_Classification_Path": str,
        "Best_LSTM_Regression_RMSE": float,
        "Best_LSTM_Regression_Path": str,
        "Best_Transformer_Classification_Accuracy": float,
        "Best_Transformer_Classification_Path": str,
        "Best_Transformer_Regression_RMSE": float,
        "Best_Transformer_Regression_Path": str,
        "Best_XGBClassifier_Classification_Accuracy": float,
        "Best_XGBClassifier_Classification_Path": str,
        "Best_XGBRegressor_Regression_RMSE": float,
        "Best_XGBRegressor_Regression_Path": str
    }


    if os.path.isfile(csv_file_path):
        # Load the existing file into a DataFrame
        ticker_df = pd.read_csv(csv_file_path)
        
        # Ensure all specified columns are present
        for column, dtype in column_types.items():
            if column not in ticker_df.columns:
                ticker_df[column] = pd.Series(dtype=dtype)
        
        # Reorder columns alphabetically, excluding 'Ticker_Symbol'
        columns = ["Ticker_Symbol"] + sorted([col for col in ticker_df.columns if col != "Ticker_Symbol"])
        ticker_df = ticker_df[columns]
    else:
        # Create a new DataFrame with the specified column types
        ticker_df = pd.DataFrame(columns=column_types.keys()).astype(column_types)
    
    return ticker_df

In [4]:
def delete_hyperparameter_search_model(ticker_symbol, model_type):
    csv_path = '../models/hyperparameters-search-models/ticker-all-models-best-hyperparameters-list.csv'
    conv1d_classification_model_path = f'../models/hyperparameters-search-models/pytorch/{model_type}/{ticker_symbol}.pth'
    conv1d_regression_model_path = f'../models/hyperparameters-search-models/pytorch/{model_type}/{ticker_symbol}.pth'
    lstm_classification_model_path = f'../models/hyperparameters-search-models/pytorch/{model_type}/{ticker_symbol}.pth'
    lstm_regression_model_path = f'../models/hyperparameters-search-models/pytorch/{model_type}/{ticker_symbol}.pth'
    
    conv1d_classification_params_path = f'../models/best-hyperparameters/pytorch/{model_type}/{ticker_symbol}.json'
    conv1d_regression_params_path = f'../models/best-hyperparameters/pytorch/{model_type}/{ticker_symbol}.json'
    lstm_classification_params_path = f'../models/best-hyperparameters/pytorch/{model_type}/{ticker_symbol}.json'
    lstm_regression_params_path = f'../models/best-hyperparameters/pytorch/{model_type}/{ticker_symbol}.json'

    if model_type == "conv1d-classification":
        if os.path.isfile(conv1d_classification_model_path):
            os.remove(conv1d_classification_model_path)
            print(f"Deleted {conv1d_classification_model_path}")
        if os.path.isfile(conv1d_classification_params_path):
            os.remove(conv1d_classification_params_path)
            print(f"Deleted {conv1d_classification_params_path}")
    
    if model_type == "conv1d-regression":
        if os.path.isfile(conv1d_regression_model_path):
            os.remove(conv1d_regression_model_path)
            print(f"Deleted {conv1d_regression_model_path}")
        if os.path.isfile(conv1d_regression_params_path):
            os.remove(conv1d_regression_params_path)
            print(f"Deleted {conv1d_regression_params_path}")

    if model_type == "lstm-classification":
        if os.path.isfile(lstm_classification_model_path):
            os.remove(lstm_classification_model_path)
            print(f"Deleted {lstm_classification_model_path}")
        if os.path.isfile(lstm_classification_params_path):
            os.remove(lstm_classification_params_path)
            print(f"Deleted {lstm_classification_params_path}")
    
    if model_type == "lstm-regression":
        if os.path.isfile(lstm_regression_model_path):
            os.remove(lstm_regression_model_path)
            print(f"Deleted {lstm_regression_model_path}")
        if os.path.isfile(lstm_regression_params_path):
            os.remove(lstm_regression_params_path)
            print(f"Deleted {lstm_regression_params_path}")

    if os.path.isfile(csv_path):
        ticker_df = pd.read_csv(csv_path)
        if ticker_symbol in ticker_df['Ticker_Symbol'].values:
            if model_type == "conv1d-classification":
                ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, ['Best_Cov1D_Classification_Accuracy', 'Best_Cov1D_Classification_Path']] = [pd.NA, pd.NA]
            if model_type == "conv1d-regression":
                ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, ['Best_CoviD_Regression_RMSE', 'Best_Cov1D_Regression_Path']] = [pd.NA, pd.NA]
            if model_type == "lstm-classification":
                ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, ['Best_LSTM_Classification_Accuracy', 'Best_LSTM_Classification_Path']] = [pd.NA, pd.NA]
            if model_type == "lstm-regression":
                ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, ['Best_LSTM_Regression_RMSE', 'Best_LSTM_Regression_Path']] = [pd.NA, pd.NA]
                
            ticker_df.to_csv(csv_path, index=False)
            print(f"Deleted {ticker_symbol} from {csv_path}")
        else:
            print(f"{ticker_symbol} not found in {csv_path}")
    else:
        print(f"{csv_path} does not exist")


In [5]:
def training_preprocess_data(df):
    #if df.isna().sum().sum() > 0 or df.isin([float('inf'), float('-inf')]).sum().sum() > 0:
     #   df = df.replace([float('inf'), float('-inf')], float('nan')).dropna()

    df.drop(df.index[:88], inplace=True)

    # Create target variables before dropping columns
    y_classifier = (np.sign(df['DAILY_MIDPRICE_CHANGE']) >= 0).astype(int)
    y_regressor = df['DAILY_MIDPRICE_CHANGE']

    # Drop columns from index 7 to 72
    #df = df.drop(df.columns[73:93], axis=1)
    #df = df.drop(df.columns[7:73], axis=1)

    # Drop specific columns
    columns_to_drop = [
        'NEXT_DAY_CLOSEPRICE', 'DAILY_CLOSEPRICE_CHANGE', 'DAILY_CLOSEPRICE_CHANGE_PERCENT', 'DAILY_CLOSEPRICE_DIRECTION',
        'DAILY_MIDPRICE', 'NEXT_DAY_MIDPRICE', 'DAILY_MIDPRICE_CHANGE', 'DAILY_MIDPRICE_CHANGE_PERCENT', 'DAILY_MIDPRICE_DIRECTION', 
        'Date', 
    ]
    df = df.drop(columns=columns_to_drop)

    X = df

    return X, y_classifier, y_regressor

In [6]:
class Conv1ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, l2_lambda=0.01, dropout_rate=0.5):
        super(Conv1ResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding='same')
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride=1, padding='same')
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=False)  # Ensure no in-place operation
        self.dropout = nn.Dropout(dropout_rate)
        
        nn.init.kaiming_normal_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv2.weight, nonlinearity='relu')
        nn.init.zeros_(self.conv1.bias)
        nn.init.zeros_(self.conv2.bias)
        
        self.l2_lambda = l2_lambda

        if in_channels != out_channels:
            self.residual_conv = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1)
        else:
            self.residual_conv = nn.Identity()

    def forward(self, x):
        residual = self.residual_conv(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = out + residual  # Ensure no in-place operation
        out = self.relu(out)
        
        return out

class Conv1DModel(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, num_blocks=1, l2_lambda=0.01, dropout_rate=0.5, classification=True):
        super(Conv1DModel, self).__init__()
        self.blocks = nn.Sequential(
            Conv1ResidualBlock(in_channels, out_channels, kernel_size, l2_lambda=l2_lambda, dropout_rate=dropout_rate),
            *[Conv1ResidualBlock(out_channels, out_channels, kernel_size, l2_lambda=l2_lambda, dropout_rate=dropout_rate) for _ in range(num_blocks-1)]
        )
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)  # Global average pooling for 1D
        self.fc = nn.Linear(out_channels, 2 if classification else 1)
        self.classification = classification

    def forward(self, x):
        out = self.blocks(x)
        out = self.global_avg_pool(out)
        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = self.fc(out)
        if self.classification:
            out = F.log_softmax(out, dim=1)
        return out

In [7]:
def conv1d_classification_hyperparameters_search(X, y, gpu_available, ticker_symbol, delete_old_data = False):
    if delete_old_data:
        delete_hyperparameter_search_model(ticker_symbol, "conv1d-classification")

    model_path = f'../models/hyperparameters-search-models/pytorch/conv1d-classification/{ticker_symbol}.pth'
    csv_path = f'../models/hyperparameters-search-models/ticker-all-models-best-hyperparameters-list.csv'
    params_path = f'../models/best-hyperparameters/pytorch/conv1d-classification/{ticker_symbol}.json'
    
    device = torch.device('cuda' if gpu_available and torch.cuda.is_available() else 'cpu')
    
    # Convert DataFrame to numpy array
    X = X.to_numpy()
    y = y.to_numpy()
    
    # Reshape X for Conv1D
    NUM_CHANNELS = 1
    X = X.reshape((X.shape[0], NUM_CHANNELS, -1))  # Reshape for Conv1D: (batch_size, num_channels, sequence_length)
    
    # Split data into training and validation sets
    TEST_SIZE = 0.2
    RANDOM_STATE = 42
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    
    def conv1d_objective(trial):
        in_channels = X_train.shape[1]
        out_channels = trial.suggest_int('out_channels', 16, 128)
        kernel_size = trial.suggest_int('kernel_size', 3, 7)
        num_blocks = trial.suggest_int('num_blocks', 1, 5)
        l2_lambda = trial.suggest_float('l2_lambda', 1e-5, 1e-2)
        dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
        epochs = trial.suggest_int('epochs', 100, 1000)
        
        model = Conv1DModel(in_channels, out_channels, kernel_size, num_blocks, l2_lambda, dropout_rate, classification=True).to(device)
        optimizer = optim.Adam(model.parameters(), lr=trial.suggest_float('lr', 1e-5, 1e-2), weight_decay=l2_lambda)
        criterion = nn.CrossEntropyLoss()
        
        patience = 10
        best_val_accuracy = -np.inf
        epochs_no_improve = 0

        input_train = torch.tensor(X_train, dtype=torch.float32).to(device)
        target_train = torch.tensor(y_train, dtype=torch.long).to(device)
        
        input_val = torch.tensor(X_val, dtype=torch.float32).to(device)
        target_val = torch.tensor(y_val, dtype=torch.long).to(device)
        
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            output = model(input_train)
            loss = criterion(output, target_train)
            loss.backward()
            optimizer.step()
            
            # Validation
            model.eval()
            with torch.no_grad():
                val_output = model(input_val)
                val_pred = val_output.argmax(dim=1)
                val_accuracy = accuracy_score(target_val.cpu(), val_pred.cpu())
                
                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                
                if epochs_no_improve >= patience:
                    print(f"Early stopping at epoch {epoch}")
                    break
        
        return best_val_accuracy  
    
    study = optuna.create_study(direction='maximize')
    study.optimize(conv1d_objective, n_trials=100)
    
    best_model = Conv1DModel(X.shape[1], study.best_params['out_channels'], study.best_params['kernel_size'], study.best_params['num_blocks'], study.best_params['l2_lambda'], study.best_params['dropout_rate'], classification=True).to(device)

    ticker_df = load_or_create_ticker_df(csv_path)
    
    # Update ticker_df and save the best model
    metric_col = 'Best_Cov1D_Classification_Accuracy'
    path_col = 'Best_Cov1D_Classification_Path'
    
    if ticker_symbol in ticker_df['Ticker_Symbol'].values:
        current_score = ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, metric_col].values[0]
        if pd.isnull(current_score) or study.best_value > current_score:
            ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, [metric_col, path_col]] = [study.best_value, model_path]
            torch.save(best_model.state_dict(), model_path)
            with open(params_path, 'w') as f:
                json.dump(study.best_params, f)
            print(f"parameters for {ticker_symbol} saved to {params_path}")
            ticker_df.to_csv(csv_path, index=False)
            print(f"Best model for {ticker_symbol} saved with accuracy: {study.best_value}")
        else:
            print(f"Previous model accuracy: {current_score} is better for {ticker_symbol} than accuracy: {study.best_value}")
    else:
        new_row = pd.DataFrame({'Ticker_Symbol': [ticker_symbol], metric_col: [study.best_value], path_col: [model_path]})
        ticker_df = pd.concat([ticker_df, new_row], ignore_index=True)
        torch.save(best_model.state_dict(), model_path)
        with open(params_path, 'w') as f:
            json.dump(study.best_params, f)
        print(f"parameters for {ticker_symbol} saved to {params_path}")
        ticker_df.to_csv(csv_path, index=False)
        print(f"Best model for {ticker_symbol} saved with accuracy: {study.best_value}")


In [8]:
def conv1d_regression_hyperparameters_search(X, y, gpu_available, ticker_symbol, delete_old_data = False):
    if delete_old_data:
        delete_hyperparameter_search_model(ticker_symbol, "conv1d-regression")

    model_path = f'../models/hyperparameters-search-models/pytorch/conv1d-regression/{ticker_symbol}.pth'
    csv_path = f'../models/hyperparameters-search-models/ticker-all-models-best-hyperparameters-list.csv'
    params_path = f'../models/best-hyperparameters/pytorch/conv1d-regression/{ticker_symbol}.json'
    
    device = torch.device('cuda' if gpu_available and torch.cuda.is_available() else 'cpu')
    
    # Convert DataFrame to numpy array
    X = X.to_numpy()
    y = y.to_numpy().reshape(-1, 1)
    
    # Reshape X for Conv1D
    NUM_CHANNELS = 1
    X = X.reshape((X.shape[0], NUM_CHANNELS, -1))  # Reshape for Conv1D: (batch_size, num_channels, sequence_length)
    
    # Split data into training and validation sets
    TEST_SIZE = 0.2
    RANDOM_STATE = 42
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    
    def conv1d_objective(trial):
        in_channels = X_train.shape[1]
        out_channels = trial.suggest_int('out_channels', 16, 128)
        kernel_size = trial.suggest_int('kernel_size', 3, 7)
        num_blocks = trial.suggest_int('num_blocks', 1, 5)
        l2_lambda = trial.suggest_float('l2_lambda', 1e-5, 1e-2)
        dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
        epochs = trial.suggest_int('epochs', 10, 100)
        
        model = Conv1DModel(in_channels, out_channels, kernel_size, num_blocks, l2_lambda, dropout_rate, classification=False).to(device)
        optimizer = optim.Adam(model.parameters(), lr=trial.suggest_float('lr', 1e-5, 1e-2), weight_decay=l2_lambda)
        criterion = nn.MSELoss()
        
        patience = 10
        best_val_rmse = np.inf
        epochs_no_improve = 0

        input_train = torch.tensor(X_train, dtype=torch.float32).to(device)
        target_train = torch.tensor(y_train, dtype=torch.float32).to(device)
        
        input_val = torch.tensor(X_val, dtype=torch.float32).to(device)
        target_val = torch.tensor(y_val, dtype=torch.float32).to(device)
        
        for epoch in range(epochs):
            model.train() 
            optimizer.zero_grad()
            output = model(input_train)
            loss = criterion(output, target_train)
            loss.backward()
            optimizer.step()
        
            model.eval()
            with torch.no_grad():
                val_output = model(input_val)
                val_rmse = root_mean_squared_error(target_val.cpu(), val_output.cpu())

                if val_rmse < best_val_rmse:
                    best_val_rmse = val_rmse
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                
                if epochs_no_improve >= patience:
                    print(f"Early stopping at epoch {epoch}")
                    break
                
        return best_val_rmse
    
    study = optuna.create_study(direction='minimize')
    study.optimize(conv1d_objective, n_trials=100)
    
    best_model = Conv1DModel(X.shape[1], study.best_params['out_channels'], study.best_params['kernel_size'], study.best_params['num_blocks'], study.best_params['l2_lambda'], study.best_params['dropout_rate'], classification=False).to(device)

    ticker_df = load_or_create_ticker_df(csv_path)
    
    # Update ticker_df and save the best model
    metric_col = 'Best_Cov1D_Regression_RMSE'
    path_col = 'Best_Cov1D_Regression_Path'
    
    if ticker_symbol in ticker_df['Ticker_Symbol'].values:
        current_score = ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, metric_col].values[0]
        if pd.isnull(current_score) or study.best_value < current_score:
            ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, [metric_col, path_col]] = [study.best_value, model_path]
            torch.save(best_model.state_dict(), model_path)
            with open(params_path, 'w') as f:
                json.dump(study.best_params, f)
            print(f"parameters for {ticker_symbol} saved to {params_path}")
            ticker_df.to_csv(csv_path, index=False)
            print(f"Best model for {ticker_symbol} saved with RMSE: {study.best_value}")
        else:
            print(f"Previous model RMSE: {current_score} is better for {ticker_symbol} than RMSE: {study.best_value}")
    else:
        new_row = pd.DataFrame({'Ticker_Symbol': [ticker_symbol], metric_col: [study.best_value], path_col: [model_path]})
        ticker_df = pd.concat([ticker_df, new_row], ignore_index=True)
        torch.save(best_model.state_dict(), model_path)
        with open(params_path, 'w') as f:
            json.dump(study.best_params, f)
        print(f"parameters for {ticker_symbol} saved to {params_path}")
        ticker_df.to_csv(csv_path, index=False)
        print(f"Best model for {ticker_symbol} saved with RMSE: {study.best_value}")

In [9]:
class LSTMResidualBlock(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout_rate=0.5):
        super(LSTMResidualBlock, self).__init__()
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout_rate if num_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout_rate)
        
        if input_size != hidden_size:
            self.residual_fc = nn.Linear(input_size, hidden_size)
        else:
            self.residual_fc = nn.Identity()

    def forward(self, x):
        residual = self.residual_fc(x)
        out, _ = self.lstm(x)
        out = self.dropout(out)
        out = out + residual  # Ensure no in-place operation
        return out

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_blocks=1, num_layers=1, dropout_rate=0.5, classification=True):
        super(LSTMModel, self).__init__()
        self.blocks = nn.Sequential(
            LSTMResidualBlock(input_size, hidden_size, num_layers=num_layers, dropout_rate=dropout_rate),
            *[LSTMResidualBlock(hidden_size, hidden_size, num_layers=num_layers, dropout_rate=dropout_rate) for _ in range(num_blocks-1)]
        )
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)  # Global average pooling for 1D
        self.fc = nn.Linear(hidden_size, 2 if classification else 1)
        self.classification = classification

    def forward(self, x):
        out = self.blocks(x)
        out = out.mean(dim=1)  # Global average pooling
        out = self.fc(out)
        if self.classification:
            out = F.log_softmax(out, dim=1)
        return out


In [10]:
def lstm_classification_hyperparameters_search(X, y, gpu_available, ticker_symbol, delete_old_data = False):
    if delete_old_data:
        delete_hyperparameter_search_model(ticker_symbol, "lstm-classification")

    model_path = f'../models/hyperparameters-search-models/pytorch/lstm-classification/{ticker_symbol}.pth'
    csv_path = f'../models/hyperparameters-search-models/ticker-all-models-best-hyperparameters-list.csv'
    params_path = f'../models/best-hyperparameters/pytorch/lstm-classification/{ticker_symbol}.json'
    
    device = torch.device('cuda' if gpu_available and torch.cuda.is_available() else 'cpu')
    
    # Convert DataFrame to numpy array
    X = X.to_numpy()
    y = y.to_numpy()
    
    def create_sequences(X, y, sequence_length):
        sequences_X, sequences_y = [], []
        for i in range(len(X) - sequence_length + 1):
            sequences_X.append(X[i:i + sequence_length])
            sequences_y.append(y[i + sequence_length - 1])
        return np.array(sequences_X), np.array(sequences_y)
    
    def lstm_objective(trial):
        sequence_length = trial.suggest_categorical('sequence_length', [5, 7, 14, 20, 30])
        
        # Create sequences
        X_seq, y_seq = create_sequences(X, y, sequence_length)
        
        # Split data into training, validation, and test sets
        TEST_SIZE = 0.2
        VAL_SIZE = 0.1
        RANDOM_STATE = 42
        
        X_train, X_temp, y_train, y_temp = train_test_split(X_seq, y_seq, test_size=TEST_SIZE + VAL_SIZE, random_state=RANDOM_STATE)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=TEST_SIZE / (TEST_SIZE + VAL_SIZE), random_state=RANDOM_STATE)
        
        input_size = X_train.shape[2]  # Number of features
        hidden_size = trial.suggest_int('hidden_size', 16, 128)
        num_layers = trial.suggest_int('num_layers', 1, 3)
        num_blocks = trial.suggest_int('num_blocks', 1, 5)
        dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
        epochs = trial.suggest_int('epochs', 100, 1000)
        
        model = LSTMModel(input_size, hidden_size, num_blocks, num_layers, dropout_rate, classification=True).to(device)
        optimizer = optim.Adam(model.parameters(), lr=trial.suggest_float('lr', 1e-5, 1e-2))
        criterion = nn.CrossEntropyLoss()
        
        patience = 10
        best_val_accuracy = -np.inf
        epochs_no_improve = 0

        input_train = torch.tensor(X_train, dtype=torch.float32).to(device)
        target_train = torch.tensor(y_train, dtype=torch.long).to(device)
        
        input_val = torch.tensor(X_val, dtype=torch.float32).to(device)
        target_val = torch.tensor(y_val, dtype=torch.long).to(device)
        
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            output = model(input_train)
            loss = criterion(output, target_train)
            loss.backward()
            optimizer.step()
            
            # Validation
            model.eval()
            with torch.no_grad():
                val_output = model(input_val)
                val_pred = val_output.argmax(dim=1)
                val_accuracy = accuracy_score(target_val.cpu(), val_pred.cpu())
                
                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                
                if epochs_no_improve >= patience:
                    print(f"Early stopping at epoch {epoch}")
                    break
        
        return best_val_accuracy  
    
    study = optuna.create_study(direction='maximize')
    study.optimize(lstm_objective, n_trials=100)
    
    # Reshape X to match the best sequence length
    best_sequence_length = study.best_params['sequence_length']
    X_seq, y_seq = create_sequences(X, y, best_sequence_length)
    
    best_model = LSTMModel(X_seq.shape[2], study.best_params['hidden_size'], study.best_params['num_blocks'], study.best_params['num_layers'], study.best_params['dropout_rate'], classification=True).to(device)

    ticker_df = load_or_create_ticker_df(csv_path)
    
    # Update ticker_df and save the best model
    metric_col = 'Best_LSTM_Classification_Accuracy'
    path_col = 'Best_LSTM_Classification_Path'
    
    if ticker_symbol in ticker_df['Ticker_Symbol'].values:
        current_score = ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, metric_col].values[0]
        if pd.isnull(current_score) or study.best_value > current_score:
            ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, [metric_col, path_col]] = [study.best_value, model_path]
            torch.save(best_model.state_dict(), model_path)
            with open(params_path, 'w') as f:
                json.dump(study.best_params, f)
            print(f"parameters for {ticker_symbol} saved to {params_path}")
            ticker_df.to_csv(csv_path, index=False)
            print(f"Best model for {ticker_symbol} saved with accuracy: {study.best_value}")
        else:
            print(f"Previous model accuracy: {current_score} is better for {ticker_symbol} than accuracy: {study.best_value}")
    else:
        new_row = pd.DataFrame({'Ticker_Symbol': [ticker_symbol], metric_col: [study.best_value], path_col: [model_path]})
        ticker_df = pd.concat([ticker_df, new_row], ignore_index=True)
        torch.save(best_model.state_dict(), model_path)
        with open(params_path, 'w') as f:
            json.dump(study.best_params, f)
        print(f"parameters for {ticker_symbol} saved to {params_path}")
        ticker_df.to_csv(csv_path, index=False)
        print(f"Best model for {ticker_symbol} saved with accuracy: {study.best_value}")

In [11]:
def lstm_regression_hyperparameters_search(X, y, gpu_available, ticker_symbol, delete_old_data = False):
    if delete_old_data:
        delete_hyperparameter_search_model(ticker_symbol, "lstm--regression")
        
    model_path = f'../models/hyperparameters-search-models/pytorch/lstm-regression/{ticker_symbol}.pth'
    csv_path = f'../models/hyperparameters-search-models/ticker-all-models-best-hyperparameters-list.csv'
    params_path = f'../models/best-hyperparameters/pytorch/lstm-regression/{ticker_symbol}.json'

    device = torch.device('cuda' if gpu_available and torch.cuda.is_available() else 'cpu')
    
    # Convert DataFrame to numpy array
    X = X.to_numpy()
    y = y.to_numpy().reshape(-1, 1)
    
    def create_sequences(X, y, sequence_length):
        sequences_X, sequences_y = [], []
        for i in range(len(X) - sequence_length + 1):
            sequences_X.append(X[i:i + sequence_length])
            sequences_y.append(y[i + sequence_length - 1])
        return np.array(sequences_X), np.array(sequences_y)
    
    def lstm_objective(trial):
        sequence_length = trial.suggest_categorical('sequence_length', [5, 7, 14, 20, 30])
        
        # Create sequences
        X_seq, y_seq = create_sequences(X, y, sequence_length)
        
        # Split data into training, validation, and test sets
        TEST_SIZE = 0.2
        VAL_SIZE = 0.1
        RANDOM_STATE = 42
        
        X_train, X_temp, y_train, y_temp = train_test_split(X_seq, y_seq, test_size=TEST_SIZE + VAL_SIZE, random_state=RANDOM_STATE)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=TEST_SIZE / (TEST_SIZE + VAL_SIZE), random_state=RANDOM_STATE)
        
        input_size = X_train.shape[2]  # Number of features
        hidden_size = trial.suggest_int('hidden_size', 16, 128)
        num_layers = trial.suggest_int('num_layers', 1, 3)
        num_blocks = trial.suggest_int('num_blocks', 1, 5)
        dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
        epochs = trial.suggest_int('epochs', 10, 100)
        
        model = LSTMModel(input_size, hidden_size, num_blocks, num_layers, dropout_rate, classification=False).to(device)
        optimizer = optim.Adam(model.parameters(), lr=trial.suggest_float('lr', 1e-5, 1e-2))
        criterion = nn.MSELoss()
        
        patience = 10
        best_val_rmse = np.inf
        epochs_no_improve = 0

        input_train = torch.tensor(X_train, dtype=torch.float32).to(device)
        target_train = torch.tensor(y_train, dtype=torch.float32).to(device)
        
        input_val = torch.tensor(X_val, dtype=torch.float32).to(device)
        target_val = torch.tensor(y_val, dtype=torch.float32).to(device)
        
        for epoch in range(epochs):
            model.train() 
            optimizer.zero_grad()
            output = model(input_train)
            loss = criterion(output, target_train)
            loss.backward()
            optimizer.step()
        
            model.eval()
            with torch.no_grad():
                val_output = model(input_val)
                val_rmse = root_mean_squared_error(target_val.cpu(), val_output.cpu())

                if val_rmse < best_val_rmse:
                    best_val_rmse = val_rmse
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                
                if epochs_no_improve >= patience:
                    print(f"Early stopping at epoch {epoch}")
                    break
                
        return best_val_rmse
    
    study = optuna.create_study(direction='minimize')
    study.optimize(lstm_objective, n_trials=100)
    
    # Reshape X to match the best sequence length
    best_sequence_length = study.best_params['sequence_length']
    X_seq, y_seq = create_sequences(X, y, best_sequence_length)
    
    best_model = LSTMModel(X_seq.shape[2], study.best_params['hidden_size'], study.best_params['num_blocks'], study.best_params['num_layers'], study.best_params['dropout_rate'], classification=False).to(device)

    ticker_df = load_or_create_ticker_df(csv_path)
    
    # Update ticker_df and save the best model
    metric_col = 'Best_LSTM_Regression_RMSE'
    path_col = 'Best_LSTM_Regression_Path'
    
    if ticker_symbol in ticker_df['Ticker_Symbol'].values:
        current_score = ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, metric_col].values[0]
        if pd.isnull(current_score) or study.best_value < current_score:
            ticker_df.loc[ticker_df['Ticker_Symbol'] == ticker_symbol, [metric_col, path_col]] = [study.best_value, model_path]
            torch.save(best_model.state_dict(), model_path)
            with open(params_path, 'w') as f:
                json.dump(study.best_params, f)
            print(f"parameters for {ticker_symbol} saved to {params_path}")
            ticker_df.to_csv(csv_path, index=False)
            print(f"Best model for {ticker_symbol} saved with RMSE: {study.best_value}")
        else:
            print(f"Previous model RMSE: {current_score} is better for {ticker_symbol} than RMSE: {study.best_value}")
    else:
        new_row = pd.DataFrame({'Ticker_Symbol': [ticker_symbol], metric_col: [study.best_value], path_col: [model_path]})
        ticker_df = pd.concat([ticker_df, new_row], ignore_index=True)
        torch.save(best_model.state_dict(), model_path)
        with open(params_path, 'w') as f:
            json.dump(study.best_params, f)
        print(f"parameters for {ticker_symbol} saved to {params_path}")
        ticker_df.to_csv(csv_path, index=False)
        print(f"Best model for {ticker_symbol} saved with RMSE: {study.best_value}")


In [12]:
for ticker_symbol in ticker_list:
    dataframe = pd.read_csv(f"../data/train/{ticker_symbol}.csv")
    X, y_classifier, y_regressor = training_preprocess_data(dataframe)
    ticker_df = conv1d_classification_hyperparameters_search(X, y_classifier, gpu_available, ticker_symbol, True)
    ticker_df = conv1d_regression_hyperparameters_search(X, y_regressor, gpu_available,ticker_symbol, True)
    ticker_df = lstm_classification_hyperparameters_search(X, y_classifier, gpu_available, ticker_symbol, True)
    ticker_df = lstm_regression_hyperparameters_search(X, y_regressor, gpu_available, ticker_symbol, True)
    #process_transformer_classification(X, y_classifier, gpu_available)
    #break

[I 2024-09-12 09:53:07,549] A new study created in memory with name: no-name-fbc0f97f-06d8-4daf-997b-cefeace86e56


Deleted ../models/hyperparameters-search-models/pytorch/conv1d-classification/CL=F.pth
Deleted ../models/best-hyperparameters/pytorch/conv1d-classification/CL=F.json
Deleted CL=F from ../models/hyperparameters-search-models/ticker-all-models-best-hyperparameters-list.csv


C:\Users\ng_mi\Anaconda\envs\pytorch-gpu\Lib\site-packages\torch\nn\modules\conv.py:304: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:1032.)
  return F.conv1d(input, weight, bias, self.stride,
[I 2024-09-12 09:53:09,481] Trial 0 finished with value: 0.512396694214876 and parameters: {'out_channels': 109, 'kernel_size': 6, 'num_blocks': 2, 'l2_lambda': 0.002357420665151709, 'dropout_rate': 0.4602281695297873, 'epochs': 410, 'lr': 0.0019425637987949625}. Best is trial 0 with value: 0.512396694214876.


Early stopping at epoch 13


[I 2024-09-12 09:53:10,965] Trial 1 finished with value: 0.512396694214876 and parameters: {'out_channels': 126, 'kernel_size': 5, 'num_blocks': 5, 'l2_lambda': 0.008803285999304479, 'dropout_rate': 0.1558615496188805, 'epochs': 896, 'lr': 0.009388985225337862}. Best is trial 0 with value: 0.512396694214876.


...

Early stopping at epoch 21


[I 2024-09-12 10:26:27,377] Trial 97 finished with value: 0.9110344648361206 and parameters: {'sequence_length': 7, 'hidden_size': 68, 'num_layers': 3, 'num_blocks': 5, 'dropout_rate': 0.49550941422466227, 'epochs': 75, 'lr': 0.008316552956588565}. Best is trial 34 with value: 0.10062485933303833.


Early stopping at epoch 32


[I 2024-09-12 10:26:28,685] Trial 98 finished with value: 0.27600589394569397 and parameters: {'sequence_length': 30, 'hidden_size': 34, 'num_layers': 1, 'num_blocks': 5, 'dropout_rate': 0.4600665510569343, 'epochs': 27, 'lr': 0.008542558335427705}. Best is trial 34 with value: 0.10062485933303833.
[I 2024-09-12 10:26:33,845] Trial 99 finished with value: 0.14449404180049896 and parameters: {'sequence_length': 14, 'hidden_size': 30, 'num_layers': 2, 'num_blocks': 5, 'dropout_rate': 0.10015385046739314, 'epochs': 97, 'lr': 0.009033650709195741}. Best is trial 34 with value: 0.10062485933303833.


parameters for USDSGD=X saved to ../models/best-hyperparameters/pytorch/lstm-regression/USDSGD=X.json
Best model for USDSGD=X saved with RMSE: 0.10062485933303833
